In [1]:
pip install gaze_tracking opencv-python


In [2]:
pip install opencv-python


Gaze tracking by webcam (murtaza)

In [ ]:
import cv2
import dlib
import numpy as np
from imutils import face_utils
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript # Import display and Javascript

# Load detector and predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

left_eye_points = [36, 37, 38, 39, 40, 41]
right_eye_points = [42, 43, 44, 45, 46, 47]

def get_eye_region(landmarks, eye_points):
    points = [landmarks.part(i) for i in eye_points]
    return np.array([(p.x, p.y) for p in points], dtype=np.int32)

# Function to take photo in Google Colab
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture Photo';
      div.append(capture);

      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      let preview = document.createElement('video');
      preview.title = 'Click to stop preview';
      preview.style.display = 'block';
      preview.srcObject = stream;
      document.body.append(div);
      div.append(preview);
      await preview.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = preview.videoWidth;
      canvas.height = preview.videoHeight;
      canvas.getContext('2d').drawImage(preview, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = base64.b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

# Import necessary functions from google.colab for the take_photo utility
from google.colab.output import eval_js
import base64

filename = take_photo() # Call the defined take_photo function
image = cv2.imread(filename)

# Check if the image was loaded successfully
if image is None:
    print(f"Error: Unable to load image from {filename}")
else:
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        landmarks = predictor(gray, face)

        for eye_points in [left_eye_points, right_eye_points]:
            region = get_eye_region(landmarks, eye_points)
            x, y, w, h = cv2.boundingRect(region)
            eye = gray[y:y+h, x:x+w]

            if eye.shape[0] == 0 or eye.shape[1] == 0:
                continue

            eye = cv2.resize(eye, (100, 50))

            # Adjusted threshold value here
            _, threshold_eye = cv2.threshold(eye, 60, 255, cv2.THRESH_BINARY_INV)

            # Show thresholded eye image for debugging
            cv2_imshow(threshold_eye)

            moments = cv2.moments(threshold_eye)

            if moments['m00'] != 0:
                cx = int(moments['m10'] / moments['m00'])

                print("Centroid x:", cx)  # Debug print

                # Adjusted left/right thresholds here
                if cx < 45:
                    direction = "Looking Left"
                elif cx > 51:
                    direction = "Looking Right"
                else:
                    direction = "Looking Center"

                cv2.putText(image, direction, (face.left(), face.top() - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2_imshow(image)